In [1076]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler



In [1077]:
data_main = pd.read_csv("bengaluru_house_prices.csv")


In [1078]:
data_main=data_main.drop(["society","area_type","availability","balcony"], axis = 1)

In [1079]:
data_main["location"]=data_main["location"].fillna("Sarjapur  Road")
data_main["size"]=data_main["size"].fillna("2 BHK")
data_main["bath"]=data_main["bath"].fillna(data_main["bath"].median())



In [1080]:
data_main.sample(15)

,location,size,total_sqft,bath,price
6908,Chandapura,2 BHK,800,2.0,25.00
3886,Dollars Layout,4 Bedroom,4200,4.0,760.00
12822,Kundalahalli,3 BHK,1724,3.0,125.00
7151,Thanisandra,2 BHK,1200,2.0,53.50
1093,Yelahanka,2 BHK,1315,2.0,85.00
5927,JP Nagar,2 BHK,1100,2.0,45.00
6438,Yelahanka,2 BHK,1519,2.0,65.00
10488,2 Bedroom Furnished Farm House in Kolar Road,2 Bedroom,1.25Acres,2.0,200.00
7357,Yelahanka,3 BHK,1650,3.0,105.00
4068,Doddanakundi Industrial Area 2,2 BHK,1512,1.0,65.01


In [1081]:
data_main["bhk"]=data_main["size"].str.split().str.get(0).astype(int)

In [1082]:
data_main.sample(12)

,location,size,total_sqft,bath,price,bhk
4501,Seegehalli,3 BHK,1830,4.0,82.0,3
11358,Sarjapur Road,4 BHK,2500,4.0,225.0,4
2231,KR Puram,2 BHK,1015,2.0,43.0,2
8417,Hongasandra,2 BHK,1180,2.0,39.0,2
13294,Sultan Palaya,4 BHK,2200,3.0,80.0,4
8893,Choodasandra,4 Bedroom,2429,3.0,210.0,4
7864,Electronic City Phase II,3 BHK,1150,3.0,46.0,3
5202,Jakkur,2 BHK,1483,2.0,98.0,2
11323,Rajaji Nagar,3 Bedroom,2300,4.0,240.0,3
8485,Thanisandra,1 BHK,662,1.0,29.0,1


In [1083]:
data_main=data_main.drop("size",axis=1)

In [1084]:
data_main

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [1085]:
data_main["location"].value_counts()

location
Whitefield                         540
Sarjapur  Road                     400
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64

In [1086]:
data_main["location"]=data_main["location"].apply(lambda x:x.strip())

In [1087]:
location_count = data_main["location"].value_counts()
location_count

location
Whitefield                              541
Sarjapur  Road                          400
Electronic City                         304
Kanakpura Road                          273
Thanisandra                             237
                                       ... 
Duddanahalli                              1
Doddanakunte                              1
Jogupalya                                 1
Subhash Nagar                             1
Kengeri Satellite Town KHB Apartment      1
Name: count, Length: 1294, dtype: int64

In [1088]:
location_10 = location_count[location_count<=11]
location_10

location
HAL 2nd Stage                           11
Kodigehalli                             11
Banjara Layout                          11
Narayanapura                            11
Thyagaraja Nagar                        11
                                        ..
Duddanahalli                             1
Doddanakunte                             1
Jogupalya                                1
Subhash Nagar                            1
Kengeri Satellite Town KHB Apartment     1
Name: count, Length: 1064, dtype: int64

In [1089]:
data_main["location"]= data_main["location"].apply(lambda x: "others" if x in location_10 else x)
data_main["location"].sample(15)


6073           Horamavu Agara
13193      Horamavu Banaswadi
6645               Kanakapura
10577    Lakshminarayana Pura
1460            Akshaya Nagar
11497                    ITPL
1390           Sarjapur  Road
1524             Malleshpalya
6983           Thigalarapalya
6202         Ramamurthy Nagar
7634            Bisuvanahalli
11801            Kundalahalli
12411                  others
11881            Bannerghatta
5105                   others
Name: location, dtype: object

In [1090]:
data_main

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,others,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [1091]:
data_main.shape

(13320, 5)

In [1092]:
# locations_reshaped = data_main['location'].values.reshape(-1, 1)
# onehot = OneHotEncoder(sparse_output=False)  
# location_encoded = onehot.fit_transform(locations_reshaped)
# location_encoded_df = pd.DataFrame(location_encoded, columns=onehot.get_feature_names_out(['location']))
# data_main = pd.concat([data_main, location_encoded_df], axis=1)
# data_main = data_main.drop('location', axis=1)


In [1093]:
data_main.sample(15)

,location,total_sqft,bath,price,bhk
11563,Varthur,1105,2.0,38.00,2
9411,Varthur,1155,2.0,39.00,2
6246,Yelahanka,1075,2.0,42.00,3
5818,Brookefield,3532,3.0,170.00,4
7146,Mallasandra,1325,2.0,65.00,2
5722,Haralur Road,1027,2.0,44.00,2
8490,others,1200,4.0,150.00,5
7698,others,1205,2.0,70.00,2
3708,others,1080,2.0,55.00,2
3458,Mallasandra,1665,3.0,86.91,3


In [1094]:
import re

def find_unit(x):
    x = str(x).strip()
    if re.match(r'^\d+(\.\d+)?$', x):
        return 'Number'
    elif '-' in x:  
        return 'Range'
    else:
        unit = re.sub(r'[\d\.\-\s]', '', x)
        return unit if unit else 'Unknown'
unit_type = data_main['total_sqft'].apply(find_unit)
print(unit_type.unique())


['Number' 'Range' 'SqMeter' 'Perch' 'SqYards' 'Acres' 'Cents' 'Guntha'
 'Grounds']


In [1095]:
data_main.sample(10)

,location,total_sqft,bath,price,bhk
11561,Mallasandra,905,2.0,35.00,2
3427,others,1650,4.0,99.00,4
8080,Raja Rajeshwari Nagar,1196,2.0,40.54,2
2531,Yeshwanthpur,1195,2.0,100.00,2
11224,Somasundara Palya,1140,2.0,46.00,2
1406,Horamavu Agara,755,2.0,37.49,2
12290,Whitefield,1220,2.0,55.00,2
12331,Billekahalli,1672,3.0,190.00,4
3401,Bannerghatta Road,1218,2.0,73.00,2
9020,others,1140,2.0,49.00,2


In [1096]:

import re

def convert_sqmt_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'sq. meter' in x_lower or 'sqmeter' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 10.7639
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x




In [1097]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_sqmt_to_sqft)

In [1098]:
data_main[3225:]

,location,total_sqft,bath,price,bhk
3225,others,1069,2.0,55.0,2
3226,Sarjapura - Attibele Road,829,2.0,22.8,2
3227,Koramangala,1900,2.0,200.0,2
3228,others,32722.26,5.0,140.0,5
3229,Sarjapur Road,1300,3.0,60.0,3
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.0,5
13316,others,3600,5.0,400.0,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.0,2
13318,Padmanabhanagar,4689,4.0,488.0,4


In [1099]:
def convert_perch_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'perch' in x_lower or '.perch' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 272.25
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x


In [1100]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_perch_to_sqft)

In [1101]:
data_main.sample(15)

,location,total_sqft,bath,price,bhk
2851,Hennur Road,1186,2.0,63.0,3
7761,Gottigere,1120,2.0,50.0,2
4187,others,2400,4.0,550.0,4
11199,Old Airport Road,1798,3.0,150.0,3
1951,Vittasandra,1238,2.0,67.0,2
939,Ramamurthy Nagar,1800,4.0,187.0,5
2395,Marathahalli,957,2.0,46.9,2
719,others,1855,3.0,225.0,3
11493,1st Phase JP Nagar,1900,3.0,400.0,4
3158,HRBR Layout,2800,3.0,250.0,3


In [1102]:
def convert_yards_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'sq. yards' in x_lower or 'sq.yards' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 9
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x


In [1103]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_yards_to_sqft)

In [1104]:
data_main[1820:]

,location,total_sqft,bath,price,bhk
1820,2nd Stage Nagarbhavi,1200,4.0,285.00,5
1821,Sarjapur,14166.0,3.0,76.00,3
1822,Nagarbhavi,884,2.0,36.00,1
1823,Haralur Road,1194,2.0,46.00,2
1824,Hoodi,1026,2.0,60.87,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,others,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [1105]:
def convert_acres_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'acres' in x_lower or '.acres' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 43560
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [1106]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_acres_to_sqft)

In [1107]:
data_main[7726:]

,location,total_sqft,bath,price,bhk
7726,Kanakpura Road,54885.6,1.0,125.0,1
7727,others,5422,6.0,1900.0,4
7728,Hennur Road,1020,2.0,47.0,2
7729,others,1000,2.0,150.0,2
7730,others,800,3.0,59.5,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.0,5
13316,others,3600,5.0,400.0,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.0,2
13318,Padmanabhanagar,4689,4.0,488.0,4


In [1108]:
def convert_cents_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'cents' in x_lower or '.cents' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 435.6
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [1109]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_cents_to_sqft)

In [1110]:
def convert_guntha_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'guntha' in x_lower or '.guntha' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 1089
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [1111]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_guntha_to_sqft)

In [1112]:
def convert_grounds_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'ground' in x_lower or '.grounds' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 2400
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [1113]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_grounds_to_sqft)

In [1114]:
data_main["total_sqft"].nunique

<bound method IndexOpsMixin.nunique of 0        1056
1        2600
2        1440
3        1521
4        1200
         ... 
13315    3453
13316    3600
13317    1141
13318    4689
13319     550
Name: total_sqft, Length: 13320, dtype: object>

In [1115]:
def find_unit(x):
    x = str(x).strip()
    if re.match(r'^\d+(\.\d+)?$', x):
        return 'Number'
    elif '-' in x:  
        return 'Range'
    else:
        unit = re.sub(r'[\d\.\-\s]', '', x)
        return unit if unit else 'Unknown'
unit_type = data_main['total_sqft'].apply(find_unit)
print(unit_type.unique())

['Number' 'Range']


In [1116]:
def convert_unit(x):
    x=str(x)
    a = x.split("-")
    if len(a) == 2:  # case: "200-300"
        return (float(a[0]) + float(a[1])) / 2
    try:  # case: "250"
        return float(a[0])
    except:  # case: "abc"
        return None


In [1117]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_unit)

In [1118]:
data_main[4894:]

,location,total_sqft,bath,price,bhk
4894,Bommanahalli,1625.0,2.0,87.72,3
4895,Whitefield,1400.0,2.0,80.00,2
4896,Electronic City,1310.0,2.0,37.83,3
4897,others,1100.0,2.0,48.00,2
4898,others,1230.0,2.0,53.51,3
...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5
13316,others,3600.0,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2
13318,Padmanabhanagar,4689.0,4.0,488.00,4


In [1119]:
data_main.describe()

,total_sqft,bath,price,bhk
count,1.332000e+04,13320.000000,13320.000000,13320.000000
mean,1.911597e+03,2.688814,112.565627,2.802778
std,1.727690e+04,1.338754,148.971674,1.294496
min,1.000000e+00,1.000000,8.000000,1.000000
25%,1.100000e+03,2.000000,50.000000,2.000000
50%,1.277250e+03,2.000000,72.000000,3.000000
75%,1.680000e+03,3.000000,120.000000,3.000000
max,1.306800e+06,40.000000,3600.000000,43.000000


In [1120]:
data_main.shape

(13320, 5)

In [1121]:
data_main = data_main[((data_main["total_sqft"]/data_main["bhk"]>=300))]

In [1122]:
data_main.shape

(12572, 5)

In [1123]:
data_main.describe()

,total_sqft,bath,price,bhk
count,1.257200e+04,12572.000000,12572.000000,12572.000000
mean,1.967604e+03,2.560770,111.483919,2.651766
std,1.778157e+04,1.081684,151.965887,0.981470
min,3.000000e+02,1.000000,8.440000,1.000000
25%,1.116000e+03,2.000000,49.000000,2.000000
50%,1.300000e+03,2.000000,70.000000,3.000000
75%,1.703250e+03,3.000000,115.000000,3.000000
max,1.306800e+06,16.000000,3600.000000,16.000000


In [1124]:
data_main['price_per_sqft'] = data_main['price'] * 100000 / data_main['total_sqft']

C:\Users\ROHIN\AppData\Local\Temp\ipykernel_3000\4172819595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_main['price_per_sqft'] = data_main['price'] * 100000 / data_main['total_sqft']


In [1125]:
data_main

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,others,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


In [1126]:
data_main["price_per_sqft"].describe()

count     12572.000000
mean       6294.446252
std        4163.245711
min           2.257423
25%        4203.059701
50%        5291.005291
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

In [1127]:


def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        
        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output, gen_df], ignore_index=True)
    return df_output

In [1128]:
data_main = remove_outliers_sqft(data_main)

In [1129]:
data_main.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10347.000000,10347.000000,10347.000000,10347.000000,10347.000000
mean,1576.866752,2.473374,91.275263,2.575626,5652.313280
std,6478.270547,0.982026,86.188437,0.900070,2241.980923
min,300.000000,1.000000,10.000000,1.000000,33.210897
25%,1109.000000,2.000000,49.000000,2.000000,4241.109085
50%,1286.000000,2.000000,67.000000,2.000000,5175.202156
75%,1650.000000,3.000000,100.000000,3.000000,6433.765573
max,653400.000000,16.000000,2200.000000,16.000000,24000.000000


In [1130]:
data_main.shape

(10347, 6)

In [1131]:
data_main.sample(15)

,location,total_sqft,bath,price,bhk,price_per_sqft
7883,Yeshwanthpur,1850.0,3.0,100.00,3,5405.405405
8449,others,1020.0,2.0,40.00,2,3921.568627
6400,Shampura,1475.0,3.0,78.00,3,5288.135593
4975,Marathahalli,1019.0,2.0,49.86,2,4893.032385
4347,Kasturi Nagar,1101.0,2.0,65.00,2,5903.723887
1741,Devanahalli,1010.0,2.0,51.00,2,5049.504950
1015,Begur,1306.0,2.0,65.00,2,4977.029096
6161,Sarjapur Road,1115.0,2.0,50.00,2,4484.304933
4461,Kengeri Satellite Town,1007.0,2.0,42.00,2,4170.804369
9108,others,800.0,2.0,55.00,2,6875.000000


In [1132]:
def bhk_outlier_remover(data_main):
    exclude_indices = np.array([])
    for location, location_df in data_main.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }

        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(
                    exclude_indices,
                    bhk_df[bhk_df.price_per_sqft < (stats['mean'])].index.values
                )
    return data_main.drop(exclude_indices, axis='index')


In [1133]:
data_main = bhk_outlier_remover(data_main)

In [1134]:
data_main.shape

(7334, 6)

In [1135]:
data_main.drop(columns="price_per_sqft", inplace=True)

In [1136]:
data_main.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [1137]:
data_main.to_csv("processed_banglore_house_dataset.csv")

In [1138]:
x_train,x_test,y_train,y_test = train_test_split(data_main.drop(columns="price"),
                                                 data_main["price"],
                                                 test_size=0.2,
                                                 random_state=0)

In [1139]:
print(x_train.shape)
print(x_test.shape)

(5867, 4)
(1467, 4)


In [1140]:
from sklearn.compose import make_column_transformer
column1 = make_column_transformer((OneHotEncoder(sparse_output=False),["location"]), remainder="passthrough")

In [1141]:
data_main

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2
...,...,...,...,...,...
10337,others,1200.0,2.0,70.0,2
10338,others,1800.0,1.0,200.0,1
10342,others,1353.0,2.0,110.0,2
10343,others,812.0,1.0,26.0,1


In [1142]:
from sklearn.pipeline import make_pipeline


In [1143]:

StandardScaler1 = StandardScaler()
lr1  = LinearRegression()
pipe1 = make_pipeline(column1,StandardScaler1,lr1)
pipe1.fit(x_train , y_train)
y_prediction = pipe1.predict(x_test)
r2_score(y_test,y_prediction)

0.6338577475305665

In [1144]:
StandardScaler1 = StandardScaler()
l1  = Lasso()
pipe2 = make_pipeline(column1,StandardScaler1,l1)
pipe2.fit(x_train , y_train)
y_prediction2 = pipe2.predict(x_test)
r2_score(y_test,y_prediction2)

0.630244236926172

In [1145]:
StandardScaler1 = StandardScaler()
r1 = Ridge()
pipe3 = make_pipeline(column1,StandardScaler1,r1)
pipe3.fit(x_train , y_train)
y_prediction3 = pipe3.predict(x_test)
r2_score(y_test,y_prediction3)

0.6338825687020672

In [1146]:
print("No Regularization",r2_score(y_test,y_prediction ) )
print("Lasso",r2_score(y_test,y_prediction2 ) )
print("Ridge",r2_score(y_test,y_prediction3 ) )

No Regularization 0.6338577475305665
Lasso 0.630244236926172
Ridge 0.6338825687020672


In [1147]:
data_main

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2
...,...,...,...,...,...
10337,others,1200.0,2.0,70.0,2
10338,others,1800.0,1.0,200.0,1
10342,others,1353.0,2.0,110.0,2
10343,others,812.0,1.0,26.0,1


In [1148]:
import pickle
pickle.dump(pipe1 , open("prediction_pickle.pkl","wb"))

Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
